In [1]:
cd ../

d:\OneDrive\Documents\Monash Research Project T2 2022\HazardousWaste


In [2]:
from Lib.WebScrapping.CorpusScraper import *

In [3]:
webpages = get_tag_corpus(["metal", "battery"], 1)

2022-08-29 06:57:45 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-08-29 06:57:45 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 37.0.1, Platform Windows-10-10.0.19044-SP0
2022-08-29 06:57:45 [scrapy.crawler] INFO: Overridden settings:
{}
2022-08-29 06:57:45 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-08-29 06:57:45 [scrapy.extensions.telnet] INFO: Telnet Password: 70dcbd18ec3ba051
2022-08-29 06:57:45 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2022-08-29 06:57:46 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpaut

Running Google Search at https://www.google.com.au/search?q=metal%20battery&start=0


2022-08-29 06:57:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ambri.com/technology/> (referer: None)
2022-08-29 06:57:50 [scrapy.core.scraper] DEBUG: Scraped from <200 https://ambri.com/technology/>
{'content': ['https://ambri.com/technology/', '  Technology: Ambri', '\r\n             Logo \r\n             Batteries for clean energy \r\n         Menu Home Benefits Business Technology Company News Careers Contact \n \n \n \n \n \n Ambri’s product is a ready-to-install DC containerized system, complete with shelves of cells, thermal management, weatherproof outer enclosure, and a battery management system (BMS), for applications that require high energy capacity, frequent cycling, long life and high efficiency. \n\n\n\n Cells are assembled onto trays and connected within a thermal enclosure to form a MWh-scale system, which is separately then coupled to the grid using standard industrial DC-AC bi-directional inverters. The system is insulated and “self-heating” when operated

Running keyword collection at https://ambri.com/technology/


2022-08-29 06:57:50 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-08-29 06:57:50 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 37.0.1, Platform Windows-10-10.0.19044-SP0
2022-08-29 06:57:50 [scrapy.crawler] INFO: Overridden settings:
{}
2022-08-29 06:57:50 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-08-29 06:57:50 [scrapy.extensions.telnet] INFO: Telnet Password: bed2e0c92496f361
2022-08-29 06:57:50 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2022-08-29 06:57:50 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpaut

...

Running keyword collection at https://www.forbes.com/sites/davidblackmon/2021/09/02/bill-gates-backed-startup-might-change-the-renewable-energy-storage-game/


2022-08-29 06:57:53 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-08-29 06:57:53 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 37.0.1, Platform Windows-10-10.0.19044-SP0
2022-08-29 06:57:53 [scrapy.crawler] INFO: Overridden settings:
{}
2022-08-29 06:57:53 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-08-29 06:57:53 [scrapy.extensions.telnet] INFO: Telnet Password: 88aa598e9b7f9edc
2022-08-29 06:57:53 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2022-08-29 06:57:53 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpaut

Running keyword collection at https://reneweconomy.com.au/bill-gates-backed-liquid-metal-battery-company-secures-a200-million/


2022-08-29 06:57:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Lithium_battery> (referer: None)
2022-08-29 06:57:54 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Lithium_battery>
{'content': ['https://en.wikipedia.org/wiki/Lithium_battery', 'Lithium battery - Wikipedia', '\n\t \n\t \n\t \n\t \n\t \n\t \n\t\t From Wikipedia, the free encyclopedia \n\t\t \n\t\t \n\t\t\n\t\t \n\t\t Jump to navigation \n\t\t Jump to search \n\t\t Non-rechargeable battery using lithium metal as anode \n This article is about disposable lithium batteries. For the common rechargeable batteries, see  Lithium-ion battery . For the less common metal-based rechargeable batteries, see  Rechargeable lithium metal battery . \n A request that this article title be changed\xa0to  Lithium metal battery  is  under discussion .  Please   this article until the discussion is closed. \n    CR2032  lithium  button cell  battery. \n    Lithium  9 volt ,  AA , a

Running keyword collection at https://en.wikipedia.org/wiki/Lithium_battery


In [4]:
for webpage in webpages:
    print(webpage.title)
    print(webpage.url)

  Technology: Ambri
https://ambri.com/technology/
Melting Metal for Energy Storage?  Liquid Metal Battery Explained - YouTube
https://www.youtube.com/watch?v=m8751tkBU_Q
A battery made of molten metals | MIT News | Massachusetts Institute of Technology
https://news.mit.edu/2016/battery-molten-metals-0112
A Decade In Development, Liquid-Metal Batteries By Ambri May Soon Change The Energy Storage Game
https://www.forbes.com/sites/davidblackmon/2021/09/02/bill-gates-backed-startup-might-change-the-renewable-energy-storage-game/

https://www.sciencedirect.com/topics/engineering/liquid-metal-battery

https://www.sciencedirect.com/topics/engineering/lithium-metal-battery
Bill Gates-backed liquid metal battery company secures $A200 million | RenewEconomy
https://reneweconomy.com.au/bill-gates-backed-liquid-metal-battery-company-secures-a200-million/
Lithium battery - Wikipedia
https://en.wikipedia.org/wiki/Lithium_battery
